In [3]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'
URL = requests.head(URL, allow_redirects=True).url
requests.get(URL)

<Response [200]>

In [4]:
from ipywidgets import IntProgress
from IPython.display import display
import tqdm
import time

def URLsFromPage(base_url = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'):
    
    #функция предназначена для извлечения ссылок на каждый товар в опеределенной категории товаров
    prgBar = IntProgress(min = 0, max = getPagesAmount(requests.get(base_url).text)) # Создаем прогрессбар
    display(prgBar)
    
    links = [] #список для ссылок на каждый товар
    
    for page in range(1, getPagesAmount(requests.get(base_url).text)+1, 1):
        
        URL = base_url + '?p=' + str(page)
        
        html = requests.get(URL).text
    
        products_list = BeautifulSoup(html).findAll('a',
                                        {'class': 'ProductCardVertical__name Link js--Link Link_type_default'})

        for product in products_list:
            links.append('https://www.citilink.ru' + BeautifulSoup.get(product, 'href') + 'properties/')
        prgBar.value += 1
        time.sleep(2)
    
    return links
    
    
def getPagesAmount(html1):
    
    pages_amount = BeautifulSoup(html1).find('a', {'class': 'PaginationWidget__page js--PaginationWidget__page PaginationWidget__page_last PaginationWidget__page-link'})

    pages_amount = BeautifulSoup.get(pages_amount, 'data-page')

    pages_amount = int(pages_amount)
    
    return pages_amount

print(0)

0


In [5]:

SSD_LINKS_LIST = URLsFromPage()
    

IntProgress(value=0, max=11)

In [6]:

len(SSD_LINKS_LIST)


483

In [7]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import time
from pyuser_agent import UA

def getHTML(URL):
    #agent = 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.3a3pre) Gecko/20100306 Firefox3.6 (.NET CLR 3.5.30729)'
    agent = UA().firefox
    html = requests.get(URL, headers={'User-Agent': agent}).text
    return html

def getSpecifications(html):
    table = BeautifulSoup(html).find('div', {'class': 'col2 xs-col3 ml-col-start-2 ml-col-3'}).findAll('div', {'class': 'Specifications__row'})
    table_2 = {}
    for t in table:
        name = t.find('div', {'class' : 'Specifications__column Specifications__column_name'}).text.strip()
        value = t.find('div', {'class' : 'Specifications__column Specifications__column_value'}).text.strip()
        if re.match('Гарантий', name):
            name = 'Гарантийное обслуживание'
        table_2[name]=value

    return table_2

print(0)

0


In [8]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import time


def getID(HTML):
    try:
        num = BeautifulSoup(HTML).find('div', {'class': 'ProductHeader__product-id'}).text
    except:
        num = BeautifulSoup(HTML).find('div', {'class': 'ProductPageUsersActivity__product-id'}).text
    num = num.strip()
    num2 = re.split(':', num)
    num = num2[1].strip()
    return num


In [33]:
def getPrice(html):
    try:
        price = int(BeautifulSoup(html).find('span', {'class': 'ProductPagePriceSection__default-price_current-price js--ProductPagePriceSection__default-price_current-price ProductPagePriceSection__default-price-value'}).text.strip().replace(' ', ''))
    except:
        try:
            price = int(BeautifulSoup(html).find('span', {'class': 'FairPricePopup__price-block_summary js--FairPricePopup__price-block_summary_current-price js--FairPricePopup__price-block_summary js--FairPricePopup__price-block_summary_current-price'}).text.strip().replace(' ', ''))
        except:
            price = 0
    return price

getPrice(html)

12890

In [19]:
import csv
from tqdm import tqdm
import pandas


SSD_LIST = []

with open("ssds_2.csv", mode="w", encoding='cp1251') as w_file:
    for link in tqdm(SSD_LINKS_LIST):
        html = getHTML(link)
        try:
            id_s = getID(html)
        except:
            time.sleep(60)
            html = getHTML(link)
            id_s = getID(html)
        price = getPrice(html)
        specifications = getSpecifications(html)
        file_writer = csv.writer(w_file, delimiter = ";", lineterminator="\r")
        to_write = {'ID': id_s, 'Price' : price, **specifications}
        SSD_LIST.append(to_write)
        to_write_mas = []
        for k,v in to_write.items():
            to_write_mas.append(k + ':' + v)
        file_writer.writerow(to_write_mas)
        time.sleep(2)

print('Program finished!')

  0%|          | 0/483 [00:01<?, ?it/s]


TypeError: can only concatenate str (not "int") to str

In [34]:

from tqdm import tqdm
import pandas


SSD_LIST = []


for link in tqdm(SSD_LINKS_LIST):
    html = getHTML(link)
    try:
        id_s = getID(html)
    except:
        time.sleep(60)
        html = getHTML(link)
        id_s = getID(html)
    price = getPrice(html)
    specifications = getSpecifications(html)
    to_write = {'ID': id_s, 'Price' : price, **specifications}
    SSD_LIST.append(to_write)
    time.sleep(2)
df = pandas.DataFrame(SSD_LIST).to_excel('some ssds_2.xlsx')
print('Program finished!')

 17%|█▋        | 81/483 [07:09<35:30,  5.30s/it]  


AttributeError: 'NoneType' object has no attribute 'text'

In [35]:
pandas.DataFrame(SSD_LIST).to_excel('some ssds_2.xlsx')

In [26]:
df

NameError: name 'df' is not defined

In [32]:
int(BeautifulSoup(html).find('span', {'class': 'FairPricePopup__price-block_summary js--FairPricePopup__price-block_summary_current-price js--FairPricePopup__price-block_summary js--FairPricePopup__price-block_summary_current-price'}).text.strip().replace(' ', ''))

12890

In [27]:
pandas.DataFrame(SSD_LIST)

,ID,Price,Брэнд,Модель,Тип жесткого диска,Объем накопителя,Форм-фактор,Интерфейс,Тип памяти NAND,Максимальная скорость чтения,...,Контроллер NAND,Скорость произвольного чтения (4KB) (IOPS),Скорость произвольной записи (4KB) (IOPS),Тип поставки,Разъем,Особенности,Поддержка NVMe,"Латентность, запись",Радиатор охлаждения,Объем кэш-памяти SSD
0,420253,2890,KINGSTON,SA400S37/480G,SSD,480 ГБ,"2.5""",SATA III,3D NAND,500 МБ/с,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,420251,1790,KINGSTON,SA400S37/240G,SSD,240 ГБ,"2.5""",SATA III,TLC,500 МБ/с,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1651622,2390,DIGMA,DGSR2512GS93T,SSD,512 ГБ,"2.5""",SATA III,3D TLC,520 МБ/с,...,Silicon Motion SM2259XT,60000,75000,rtl,NaN,NaN,NaN,NaN,NaN,NaN
3,1651620,1290,DIGMA,DGSR2256GS93T,SSD,256 ГБ,"2.5""",SATA III,3D TLC,510 МБ/с,...,Silicon Motion SM2259XT,50000,73000,rtl,NaN,NaN,NaN,NaN,NaN,NaN
4,1740428,1290,KINGSPEC,P3-256,SSD,256 ГБ,"2.5""",SATA III,NaN,550 МБ/с,...,NaN,70600,74400,NaN,SATA,эта модель представлена в 2-х цветах,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1121163,2390,A-DATA,AGAMMIXS5-256GT-C,SSD,256 ГБ,M.2 2280,PCI-E x4,3D TLC,2100 МБ/с,...,NaN,190000,180000,NaN,M.2,NaN,есть,NaN,есть,NaN
61,1738041,1590,DATO,DS700SSD-256GB,SSD,256 ГБ,"2.5""",SATA III,3D NAND TLC,545 МБ/с,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,1824997,7990,SAMSUNG,MZ-V8V1T0BW,SSD,1 ТБ,M.2 2280,PCI-E x4,V-NAND 3-bit MLC,3500 МБ/с,...,Samsung Pablo Controller,500000,480000,NaN,M.2,NaN,есть,NaN,NaN,NaN
63,1618423,1590,DIGMA,DGSM3256GS33T,SSD,256 ГБ,M.2 2280,PCI-E x4,3D TLC,2040 МБ/с,...,Silicon Motion SM2263XT,75000,90000,rtl,NaN,NaN,есть,NaN,NaN,NaN
